In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.ops import unary_union

from shapely import wkt
from folium.features import GeoJsonTooltip

# POPULATION

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
from shapely import wkt
import matplotlib.pyplot as plt

# ---------------------------
# 1. Read and prepare lines data
# ---------------------------
raw_df = pd.read_csv('data\\dev_wings_agg_span_2024_01_01.csv')
df_lines = raw_df[['shape', 'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
                     'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport', 'cust_total',
                     'downstream_cust_industrial', 'downstream_cust_commercial', 'downstream_cust_residential',
                     'downstream_cust_sensitive', 'downstream_cust_essential', 'downstream_cust_medicalcert',
                     'downstream_cust_urgent', 'downstream_cust_lifesupport', 'downstream_cust_total']]

# Convert the 'shape' column (WKT strings) to geometry objects
df_lines["geometry"] = df_lines["shape"].apply(wkt.loads)

gdf_lines = gpd.GeoDataFrame(df_lines, geometry="geometry", crs="EPSG:2230")

gdf_lines = gdf_lines.to_crs("EPSG:4326")

# ---------------------------
# 2. Read and prepare polygon data
# ---------------------------
subdiv_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUSUB/tl_2020_06_cousub.zip"
gdf_subdiv = gpd.read_file(subdiv_url)
gdf_sd = gdf_subdiv[gdf_subdiv['COUNTYFP'] == '073']

places_url = "https://www2.census.gov/geo/tiger/TIGER2020/PLACE/tl_2020_06_place.zip"
gdf_places = gpd.read_file(places_url)
counties_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
gdf_counties = gpd.read_file(counties_url)
gdf_sd_county = gdf_counties[gdf_counties['NAME'] == 'San Diego']

# Project places and counties to EPSG:4326
gdf_places = gdf_places.to_crs("EPSG:4326")
gdf_sd_county = gdf_sd_county.to_crs("EPSG:4326")

gdf_sd_places = gpd.sjoin(gdf_places, gdf_sd_county, how="inner", predicate="intersects")

# Remove any pre-existing 'index_right' column to avoid conflicts
if 'index_right' in gdf_sd_places.columns:
    gdf_sd_places = gdf_sd_places.drop(columns=['index_right'])

# ---------------------------
# 3. Spatial join: Accumulate customer values for lines within polygons
# ---------------------------
lines_in_polygons_within = gpd.sjoin(gdf_lines, gdf_sd_places, how="inner", predicate="within")
print("Number of matched lines with 'within':", len(lines_in_polygons_within))

if len(lines_in_polygons_within) == 0:
    lines_in_polygons = gpd.sjoin(gdf_lines, gdf_sd_places, how="inner", predicate="intersects")
    print("Number of matched lines with 'intersects':", len(lines_in_polygons))
else:
    lines_in_polygons = lines_in_polygons_within

customer_fields = [
    'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
    'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport', 'cust_total'
]

accumulated = lines_in_polygons.groupby("index_right")[customer_fields].sum()

gdf_sd_places = gdf_sd_places.merge(accumulated, left_index=True, right_index=True, how='left')

gdf_sd_places[customer_fields] = gdf_sd_places[customer_fields].fillna(0)

# ---------------------------
# 4. Create a Folium map
# ---------------------------
center = [gdf_sd_places.geometry.centroid.y.mean(), gdf_sd_places.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=10)

tooltip = GeoJsonTooltip(
    fields=customer_fields,
    aliases=[field.replace("cust_", "").capitalize() for field in customer_fields],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 1px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    gdf_sd_places,
    tooltip=tooltip,
).add_to(m)

# Uncomment to save the folium map to an HTML file
# m.save("map.html")

# ---------------------------
# 5. Save the accumulated polygon data to CSV
# ---------------------------
gdf_sd_places["geometry"] = gdf_sd_places["geometry"].apply(lambda x: x.wkt)
gdf_sd_places.to_csv("accumulated_polygon_values.csv", index=False)

m

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely import wkt

# ---------------------------
# 1. Load and prepare lines data
# ---------------------------
raw_df = pd.read_csv('data\\dev_wings_agg_span_2024_01_01.csv')
df_lines = raw_df[['shape', 'cust_industrial', 'cust_commercial', 'cust_residential', 
                     'cust_sensitive', 'cust_essential', 'cust_medicalcert', 'cust_urgent', 
                     'cust_lifesupport', 'cust_total', 'downstream_cust_industrial', 
                     'downstream_cust_commercial', 'downstream_cust_residential', 
                     'downstream_cust_sensitive', 'downstream_cust_essential', 
                     'downstream_cust_medicalcert', 'downstream_cust_urgent', 
                     'downstream_cust_lifesupport', 'downstream_cust_total']]

df_lines["geometry"] = df_lines["shape"].apply(wkt.loads)

initial_crs = "EPSG:2230"
gdf_lines = gpd.GeoDataFrame(df_lines, geometry="geometry", crs=initial_crs)

# ---------------------------
# 2. Check bounds before and after reprojection
# ---------------------------
print("Original bounds (in {}):".format(initial_crs), gdf_lines.total_bounds)
gdf_lines = gdf_lines.to_crs("EPSG:4326")
print("Reprojected bounds (EPSG:4326):", gdf_lines.total_bounds)

# ---------------------------
# 3. Sample 50,000 random line
# ---------------------------
if len(gdf_lines) > 50000:
    sample_gdf = gdf_lines.sample(n=50000, random_state=42)
else:
    sample_gdf = gdf_lines.copy()

# ---------------------------
# 4. Create a Folium map with the sampled lines
# ---------------------------
center = [sample_gdf.geometry.centroid.y.mean(), sample_gdf.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=10)

folium.GeoJson(sample_gdf.__geo_interface__, name="Random Lines").add_to(m)
folium.LayerControl().add_to(m)

m.save("sampled_lines_map.html")
m

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
from shapely import wkt
import matplotlib.pyplot as plt
import branca.colormap as cm

def risk_style(feature):
    rank = feature['properties']['risk_rank']
    return {
        'fillColor': colormap(rank),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    }

# ---------------------------
# 1. Load and prepare lines data
# ---------------------------
raw_df = pd.read_csv('data\\dev_wings_agg_span_2024_01_01.csv')
df_lines = raw_df[['shape', 'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
                     'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport', 'cust_total',
                     'downstream_cust_industrial', 'downstream_cust_commercial', 'downstream_cust_residential',
                     'downstream_cust_sensitive', 'downstream_cust_essential', 'downstream_cust_medicalcert',
                     'downstream_cust_urgent', 'downstream_cust_lifesupport', 'downstream_cust_total']]

df_lines["geometry"] = df_lines["shape"].apply(wkt.loads)

gdf_lines = gpd.GeoDataFrame(df_lines, geometry="geometry", crs="EPSG:2230")
gdf_lines = gdf_lines.to_crs("EPSG:4326")

# ---------------------------
# 2. Load and prepare polygon data
# ---------------------------
subdiv_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUSUB/tl_2020_06_cousub.zip"
gdf_subdiv = gpd.read_file(subdiv_url)
gdf_sd = gdf_subdiv[gdf_subdiv['COUNTYFP'] == '073']
gdf_sd = gdf_sd.to_crs("EPSG:4326")

places_url = "https://www2.census.gov/geo/tiger/TIGER2020/PLACE/tl_2020_06_place.zip"
gdf_places = gpd.read_file(places_url)
counties_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
gdf_counties = gpd.read_file(counties_url)
gdf_sd_county = gdf_counties[gdf_counties['NAME'] == 'San Diego']

gdf_places = gdf_places.to_crs("EPSG:4326")
gdf_sd_county = gdf_sd_county.to_crs("EPSG:4326")

gdf_sd_places = gpd.sjoin(gdf_places, gdf_sd_county, how="inner", predicate="intersects")
if 'index_right' in gdf_sd_places.columns:
    gdf_sd_places = gdf_sd_places.drop(columns=['index_right'])
gdf_sd_places = gdf_sd_places.loc[:, ~gdf_sd_places.columns.duplicated()]

# ---------------------------
# 3. Create non-overlapping background polygons
# ---------------------------
places_union = gdf_sd_places.unary_union

def subtract_places(geom, union_geom):
    return geom.difference(union_geom)

gdf_bg_nonoverlap = gdf_sd.copy()
gdf_bg_nonoverlap["geometry"] = gdf_bg_nonoverlap.geometry.apply(lambda g: subtract_places(g, places_union))
gdf_bg_nonoverlap = gdf_bg_nonoverlap[~gdf_bg_nonoverlap.is_empty]
gdf_bg_nonoverlap["layer"] = "Background"
gdf_sd_places["layer"] = "Incorporated Places"

# ---------------------------
# 4. Accumulate customer values into the non-overlapping polygons
# ---------------------------
gdf_combined = pd.concat([gdf_bg_nonoverlap, gdf_sd_places], ignore_index=True)

lines_in_polygons = gpd.sjoin(gdf_lines, gdf_combined, how="inner", predicate="intersects")

customer_fields = [
    'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
    'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport', 'cust_total', 
    'downstream_cust_industrial', 'downstream_cust_commercial', 'downstream_cust_residential',
    'downstream_cust_sensitive', 'downstream_cust_essential', 'downstream_cust_medicalcert',
    'downstream_cust_urgent', 'downstream_cust_lifesupport', 'downstream_cust_total'
]

accumulated = lines_in_polygons.groupby("index_right")[customer_fields].sum()
gdf_combined = gdf_combined.merge(accumulated, left_index=True, right_index=True, how='left')
gdf_combined[customer_fields] = gdf_combined[customer_fields].fillna(0)

# ---- Compute PSPS Risk Score and Rank ----
weights = {
    'cust_lifesupport': 10,
    'cust_medicalcert': 8,
    'cust_urgent': 6,
    'cust_sensitive': 4,
    'cust_industrial': 3,
    'cust_residential': 2,
    'cust_commercial': 1
}

gdf_combined['psps_risk'] = (
    weights['cust_lifesupport'] * (gdf_combined['cust_lifesupport'] + gdf_combined['downstream_cust_lifesupport']) +
    weights['cust_medicalcert'] * (gdf_combined['cust_medicalcert'] + gdf_combined['downstream_cust_medicalcert']) +
    weights['cust_urgent'] * (gdf_combined['cust_urgent'] + gdf_combined['downstream_cust_urgent']) +
    weights['cust_sensitive'] * (gdf_combined['cust_sensitive'] + gdf_combined['downstream_cust_sensitive']) +
    weights['cust_industrial'] * (gdf_combined['cust_industrial'] + gdf_combined['downstream_cust_industrial']) +
    weights['cust_residential'] * (gdf_combined['cust_residential'] + gdf_combined['downstream_cust_residential']) +
    weights['cust_commercial'] * (gdf_combined['cust_commercial'] + gdf_combined['downstream_cust_commercial'])
)

# Rank polygons by risk: rank 1 = highest risk.
gdf_combined['risk_rank'] = gdf_combined['psps_risk'].rank(method='min', ascending=False).astype(int)

# ---------------------------
# 5. Create the Folium Map using heatmap styling (based on risk rank)
# ---------------------------
gdf_combined_bg = gdf_combined[gdf_combined["layer"] == "Background"]
gdf_combined_places = gdf_combined[gdf_combined["layer"] == "Incorporated Places"]

min_rank = gdf_combined['risk_rank'].min()  # typically 1
max_rank = gdf_combined['risk_rank'].max()
colormap = cm.LinearColormap(colors=["red", "green"], vmin=min_rank, vmax=max_rank)
colormap.caption = "PSPS Risk Rank (1 = Highest Risk)"

center = [gdf_sd_places.geometry.unary_union.centroid.y,
          gdf_sd_places.geometry.unary_union.centroid.x]
m = folium.Map(location=center, zoom_start=10)
colormap.add_to(m)

background_tooltip = folium.GeoJsonTooltip(
    fields=['NAME', 'cust_total', 'downstream_cust_total', 'risk_rank'],
    aliases=['Section:', 'Customer Total:', 'Downstream Total:', 'Population Risk Rank:'],
    localize=True
)

folium.GeoJson(
    gdf_combined_bg.to_json(),
    name='Background Polygons',
    tooltip=background_tooltip,
    style_function=risk_style
).add_to(m)

places_tooltip = folium.GeoJsonTooltip(
    fields=['NAME_left', 'cust_total', 'downstream_cust_total', 'risk_rank'],
    aliases=['Place:', 'Customer Total:', 'Downstream Total:', 'Population Risk Rank:'],
    localize=True
)

folium.GeoJson(
    gdf_combined_places.to_json(),
    name='Incorporated Places',
    tooltip=places_tooltip,
    style_function=risk_style
).add_to(m)

folium.LayerControl().add_to(m)

# ---------------------------
# 6. Save the accumulated polygon data to CSV
# ---------------------------
gdf_combined["geometry"] = gdf_combined["geometry"].apply(lambda x: x.wkt)
gdf_combined.to_csv("accumulated_nonoverlap_polygon_values.csv", index=False)

m.save("nonoverlap_polygons_map.html")
m

# OVERALL

In [ ]:
life = life[['NAME', 'NAME_left', 'geometry', 'layer', 'psps_risk', 'risk_rank']]
life['combined_name'] = life['NAME'].fillna(life['NAME_left'])
life.drop(columns=['NAME', 'NAME_left'])

In [ ]:
energy_df = pd.read_csv("/mnt/data/Energy_Output.csv")          # expected columns: latitude, longitude, energy_score
weather_df = pd.read_csv("/mnt/data/weather_predictions.csv")    # expected columns: latitude, longitude, weather_score
life_risk_df = pd.read_csv("/mnt/data/life_risk.csv")  

#Subdiv Polygons
subdiv_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUSUB/tl_2020_06_cousub.zip"
gdf_subdiv = gpd.read_file(subdiv_url)
gdf_sd = gdf_subdiv[gdf_subdiv['COUNTYFP'] == '073']


places_url = "https://www2.census.gov/geo/tiger/TIGER2020/PLACE/tl_2020_06_place.zip"
gdf_places = gpd.read_file(places_url)
counties_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
gdf_counties = gpd.read_file(counties_url)
gdf_sd_county = gdf_counties[gdf_counties['NAME'] == 'San Diego']
gdf_places = gdf_places.to_crs("EPSG:4326")
gdf_sd_county = gdf_sd_county.to_crs("EPSG:4326")
gdf_sd_places = gpd.sjoin(gdf_places, gdf_sd_county, how="inner", predicate="intersects")

# Creating the Map

In [ ]:
energy_gpd = gpd.GeoDataFrame(energy, 
                                geometry=gpd.points_from_xy(energy['longitude'], energy['latitude']),
                                crs='EPSG:4326')

nature_gpd = gpd.GeoDataFrame(nature, 
                                geometry=gpd.points_from_xy(nature['longitude'], nature['latitude']),
                                crs='EPSG:4326')
weather_gpd = gpd.GeoDataFrame(weather, 
                                geometry=gpd.points_from_xy(weather['Longitude'], weather['Latitude']),
                                crs='EPSG:4326')

In [ ]:
# Random risk score
def custom_overall_risk(row):
    energy = row['avg_risk_gdf1']
    nature = row['avg_risk_gdf2']
    weather = row['avg_risk_gdf3']
    return 0.4 * energy + 0.3 * nature + 0.3 * weather

def calculate_avg_risk(polygon_gdf, risk_gdf, risk_field, suffix):
    """
    Performs a spatial join of polygon_gdf with risk_gdf,
    calculates the average risk score (risk_field) for each polygon,
    and returns a Series indexed by the polygon's index.
    """
    if 'index_right' in polygon_gdf.columns:
        polygon_gdf = polygon_gdf.drop(columns=['index_right'])
    if 'index_right' in risk_gdf.columns:
        risk_gdf = risk_gdf.drop(columns=['index_right'])
    
    joined = gpd.sjoin(polygon_gdf, risk_gdf, predicate='contains', how='left')
    # Group by the polygon's index (sjoin returns it as 'index_left')
    avg_risk = joined.groupby(joined.index)[risk_field].mean().rename(f'avg_risk_{suffix}')
    return avg_risk

# ---- Incorporated Places ----
for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    if col in gdf_sd_places.columns:
        gdf_sd_places.drop(columns=[col], inplace=True)

avg_risk_1_places = calculate_avg_risk(gdf_sd_places, energy_gpd, 'probability (%)', 'gdf1')
avg_risk_2_places = calculate_avg_risk(gdf_sd_places, nature_gpd, 'nature_index', 'gdf2')
avg_risk_3_places = calculate_avg_risk(gdf_sd_places, weather_gpd, 'Predicted_Fire_Risk', 'gdf3')

gdf_sd_places = gdf_sd_places.join(avg_risk_1_places, how='left', rsuffix='_new1')
gdf_sd_places = gdf_sd_places.join(avg_risk_2_places, how='left', rsuffix='_new2')
gdf_sd_places = gdf_sd_places.join(avg_risk_3_places, how='left', rsuffix='_new3')

for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    disp_col = col + '_disp'
    gdf_sd_places[disp_col] = gdf_sd_places[col].apply(lambda x: f"{x} (missing)" if pd.isna(x) else x)
    gdf_sd_places[col] = gdf_sd_places[col].fillna(0)

gdf_sd_places['overall_risk'] = gdf_sd_places.apply(custom_overall_risk, axis=1)

# ---- Background Polygons ----
for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    if col in gdf_sd.columns:
        gdf_sd.drop(columns=[col], inplace=True)

avg_risk_1_sd = calculate_avg_risk(gdf_sd, energy_gpd, 'probability (%)', 'gdf1')
avg_risk_2_sd = calculate_avg_risk(gdf_sd, nature_gpd, 'nature_index', 'gdf2')
avg_risk_3_sd = calculate_avg_risk(gdf_sd, weather_gpd, 'Predicted_Fire_Risk', 'gdf3')

gdf_sd = gdf_sd.join(avg_risk_1_sd).join(avg_risk_2_sd).join(avg_risk_3_sd)

for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    disp_col = col + '_disp'
    gdf_sd[disp_col] = gdf_sd[col].apply(lambda x: f"{x} (missing)" if pd.isna(x) else x)
    gdf_sd[col] = gdf_sd[col].fillna(0)

gdf_sd['overall_risk'] = gdf_sd.apply(custom_overall_risk, axis=1)

places_union = unary_union(gdf_sd_places.geometry)
gdf_sd['geometry'] = gdf_sd.geometry.apply(lambda geom: geom.difference(places_union))

# Remove any duplicated columns before converting to JSON
gdf_sd_places = gdf_sd_places.loc[:, ~gdf_sd_places.columns.duplicated()]
gdf_sd = gdf_sd.loc[:, ~gdf_sd.columns.duplicated()]

from branca.colormap import LinearColormap

# Compute global min and max risk from both GeoDataFrames
global_min_risk = min(gdf_sd['overall_risk'].min(), gdf_sd_places['overall_risk'].min())
global_max_risk = max(gdf_sd['overall_risk'].max(), gdf_sd_places['overall_risk'].max())
if pd.isna(global_min_risk) or global_min_risk is None:
    global_min_risk = 0
if pd.isna(global_max_risk) or global_max_risk is None:
    global_max_risk = 1

colormap = LinearColormap(['green', 'red'], vmin=global_min_risk, vmax=global_max_risk)
colormap.caption = 'Overall Risk Heatmap'

def heat_style(feature, fill_opacity=0.5):
    risk = feature['properties'].get('overall_risk')
    # If risk is missing or zero, color it gray
    if risk is None or pd.isna(risk) or risk == 0:
        fillColor = 'gray'
    else:
        fillColor = colormap(risk)
    return {
        'fillColor': fillColor,
        'color': 'black',
        'weight': 2,
        'fillOpacity': fill_opacity
    }

center = [gdf_sd_places.geometry.unary_union.centroid.y,
          gdf_sd_places.geometry.unary_union.centroid.x]
m = folium.Map(location=center, zoom_start=10)
colormap.add_to(m) 

# ---- Background Polygons ----
gdf_sd = gdf_sd.loc[:, ~gdf_sd.columns.duplicated()]

background_tooltip = folium.GeoJsonTooltip(
    fields=['NAME', 'avg_risk_gdf1_disp', 'avg_risk_gdf2_disp', 'avg_risk_gdf3_disp', 'overall_risk'],
    aliases=['Section:', 'Energy Risk:', 'Nature Risk:', 'Weather Risk:', 'Overall Risk:'],
    localize=True
)

folium.GeoJson(
    gdf_sd.to_json(),
    name='Background Polygons',
    tooltip=background_tooltip,
    style_function=lambda feature: heat_style(feature, fill_opacity=0.5)
).add_to(m)

# ---- Incorporated Places ----
gdf_sd_places = gdf_sd_places.loc[:, ~gdf_sd_places.columns.duplicated()]

places_tooltip = folium.GeoJsonTooltip(
    fields=['NAME_left', 'avg_risk_gdf1_disp', 'avg_risk_gdf2_disp', 'avg_risk_gdf3_disp', 'overall_risk'],
    aliases=['Place:', 'Energy Risk:', 'Nature Risk:', 'Weather Risk:', 'Overall Risk:'],
    localize=True
)

folium.GeoJson(
    gdf_sd_places.to_json(),
    name='Incorporated Places',
    tooltip=places_tooltip,
    style_function=lambda feature: heat_style(feature, fill_opacity=0.7)
).add_to(m)

folium.LayerControl().add_to(m)

m


In [ ]:
m.save("alpha_map.html")

# Final Product

In [ ]:
energy = pd.read_csv('outputs/Energy_Output.csv').drop(columns=['Unnamed: 0'])
nature = pd.read_csv('outputs/nature_index_no_hftd.csv')
weather = pd.read_csv('outputs/weather_predictions.csv')
life = pd.read_csv('life_risk.csv')

#Subdiv Polygons
subdiv_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUSUB/tl_2020_06_cousub.zip"
gdf_subdiv = gpd.read_file(subdiv_url)
gdf_sd = gdf_subdiv[gdf_subdiv['COUNTYFP'] == '073']


places_url = "https://www2.census.gov/geo/tiger/TIGER2020/PLACE/tl_2020_06_place.zip"
gdf_places = gpd.read_file(places_url)
counties_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
gdf_counties = gpd.read_file(counties_url)
gdf_sd_county = gdf_counties[gdf_counties['NAME'] == 'San Diego']
gdf_places = gdf_places.to_crs("EPSG:4326")
gdf_sd_county = gdf_sd_county.to_crs("EPSG:4326")
gdf_sd_places = gpd.sjoin(gdf_places, gdf_sd_county, how="inner", predicate="intersects")

In [ ]:
import numpy as np

# Define capping threshold (e.g., 95th percentile)
upper_cap = life["psps_risk"].quantile(0.95)  # Adjust threshold if needed
lower_cap = life["psps_risk"].quantile(0)  # Optional lower cap if there are small outliers

# Clip values to avoid extreme outliers
life["psps_risk_capped"] = np.clip(life["psps_risk"], lower_cap, upper_cap)

# Scale using the new min/max
min_risk = life["psps_risk_capped"].min()
max_risk = life["psps_risk_capped"].max()
life["psps_risk_scaled"] = ((life["psps_risk_capped"] - min_risk) / (max_risk - min_risk)) * 100

# Display results
life[["psps_risk", "psps_risk_capped", "psps_risk_scaled"]]


In [ ]:
upper_cap = weather["Predicted_Fire_Risk"].quantile(0.95)  # Adjust threshold if needed
lower_cap = weather["Predicted_Fire_Risk"].quantile(0)  # Optional lower cap if there are small outliers

# Clip values to avoid extreme outliers
weather["Predicted_Fire_Risk_capped"] = np.clip(weather["Predicted_Fire_Risk"], lower_cap, upper_cap)

# Scale using the new min/max
min_risk = weather["Predicted_Fire_Risk_capped"].min()
max_risk = weather["Predicted_Fire_Risk_capped"].max()
weather["Predicted_Fire_Risk_scaled"] = ((weather["Predicted_Fire_Risk_capped"] - min_risk) / (max_risk - min_risk)) * 100

# Display results
weather[["Predicted_Fire_Risk", "Predicted_Fire_Risk_capped", "Predicted_Fire_Risk_scaled"]]

In [ ]:
upper_cap = nature["nature_index"].quantile(0.95)  # Adjust threshold if needed
lower_cap = nature["nature_index"].quantile(0.05)  # Optional lower cap if there are small outliers

# Clip values to avoid extreme outliers
nature["nature_index_capped"] = np.clip(nature["nature_index"], lower_cap, upper_cap)

# Scale using the new min/max
min_risk = nature["nature_index_capped"].min()
max_risk = nature["nature_index_capped"].max()
nature["nature_index_scaled"] = ((nature["nature_index_capped"] - min_risk) / (max_risk - min_risk)) * 100

# Display results
nature[["nature_index", "nature_index_capped", "nature_index_scaled"]]

In [ ]:
weather['Predicted_Fire_Risk'].min()

In [ ]:
energy_gpd = gpd.GeoDataFrame(energy, 
                                geometry=gpd.points_from_xy(energy['longitude'], energy['latitude']),
                                crs='EPSG:4326')

nature_gpd = gpd.GeoDataFrame(nature, 
                                geometry=gpd.points_from_xy(nature['longitude'], nature['latitude']),
                                crs='EPSG:4326')
weather_gpd = gpd.GeoDataFrame(weather, 
                                geometry=gpd.points_from_xy(weather['longitude'], weather['latitude']),
                                crs='EPSG:4326')

life['geometry'] = life['geometry'].apply(wkt.loads)
life_gdf = gpd.GeoDataFrame(life, geometry='geometry', crs='EPSG:4326')

In [ ]:
import math

energy = pd.read_csv('outputs/Energy_Output.csv').drop(columns=['Unnamed: 0'])
nature = pd.read_csv('outputs/nature_index_no_hftd.csv')
weather = pd.read_csv('outputs/weather_predictions.csv')
life = pd.read_csv('life_risk.csv')

#Subdiv Polygons
subdiv_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUSUB/tl_2020_06_cousub.zip"
gdf_subdiv = gpd.read_file(subdiv_url)
gdf_sd = gdf_subdiv[gdf_subdiv['COUNTYFP'] == '073']


places_url = "https://www2.census.gov/geo/tiger/TIGER2020/PLACE/tl_2020_06_place.zip"
gdf_places = gpd.read_file(places_url)
counties_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
gdf_counties = gpd.read_file(counties_url)
gdf_sd_county = gdf_counties[gdf_counties['NAME'] == 'San Diego']
gdf_places = gdf_places.to_crs("EPSG:4326")
gdf_sd_county = gdf_sd_county.to_crs("EPSG:4326")
gdf_sd_places = gpd.sjoin(gdf_places, gdf_sd_county, how="inner", predicate="intersects")

upper_cap = life["psps_risk"].quantile(0.95)  # Adjust threshold if needed
lower_cap = life["psps_risk"].quantile(0)  # Optional lower cap if there are small outliers

# Clip values to avoid extreme outliers
life["psps_risk_capped"] = np.clip(life["psps_risk"], lower_cap, upper_cap)

# Scale using the new min/max
min_risk = life["psps_risk_capped"].min()
max_risk = life["psps_risk_capped"].max()
life["psps_risk_scaled"] = ((life["psps_risk_capped"] - min_risk) / (max_risk - min_risk)) * 100

# Display results
life[["psps_risk", "psps_risk_capped", "psps_risk_scaled"]]

import math

def composite_risk(row):
    """
    Calculate a composite risk score where the wildfire indicators
    (Nature, Weather, Energy) battle against the Life factor, without using minimum values.

    The formula is:
    
        composite = [ W_w * exp(W / λ)
                    + W_n * exp(N / λ)
                    + W_e * exp(E / λ) ]
                    - g * (L^2)
                    
    where:
      - W, N, E represent the weather, nature, and energy risk values.
      - L represents the life factor (population or PSPS indicator).
      - W_w, W_n, W_e are weights for each risk indicator.
      - λ (lambda_) is an exponential scale parameter.
      - g is the scaling factor for the life term.
      
    Adjust the parameters as needed based on your data scales.
    """
    
    # Extract risk values from the row
    W = row['avg_risk_gdf3']  # Weather risk
    N = row['avg_risk_gdf2']  # Nature risk
    E = row['avg_risk_gdf1']  # Energy risk
    L = row['avg_risk_gdf4']  # Life factor (population)
    
    # --- Parameter initialization (tune these as needed) ---
    W_w = 0.02  # Weight for weather risk
    W_n = 0.03  # Weight for nature risk
    W_e = 0.02  # Weight for energy risk
    g   = 1.0  # Scaling factor for the life term (L^2)
    
    # Exponential scale parameter: controls the amplification of risk values
    lambda_ = 1
    
    # --- Calculate the cumulative wildfire risk ---
    wildfire_sum = (
          W_w * math.exp(W / lambda_) +
          W_n * math.exp(N / lambda_) +
          W_e * math.exp(E / lambda_)
    )
    
    # --- Composite risk: wildfire risk "battles" against life ---
    composite = wildfire_sum - g * (L ** 2)
    
    return composite

# Function to calculate the average risk for a given polygon layer and risk layer
def calculate_avg_risk(polygon_gdf, risk_gdf, risk_field, suffix):
    """
    Performs a spatial join of polygon_gdf with risk_gdf,
    calculates the average risk score (risk_field) for each polygon,
    and returns a Series indexed by the polygon's index.
    """
    if 'index_right' in polygon_gdf.columns:
        polygon_gdf = polygon_gdf.drop(columns=['index_right'])
    if 'index_right' in risk_gdf.columns:
        risk_gdf = risk_gdf.drop(columns=['index_right'])
    
    joined = gpd.sjoin(polygon_gdf, risk_gdf, predicate='contains', how='left')
    # Group by the polygon's index (sjoin returns it as 'index_left')
    avg_risk = joined.groupby(joined.index)[risk_field].mean().rename(f'avg_risk_{suffix}')
    return avg_risk

# ---- Incorporated Places ----
# Remove risk columns if they exist to avoid duplicate names
for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    if col in gdf_sd_places.columns:
        gdf_sd_places.drop(columns=[col], inplace=True)

avg_risk_1_places = calculate_avg_risk(gdf_sd_places, energy_gpd, 'probability (%)', 'gdf1')
avg_risk_2_places = calculate_avg_risk(gdf_sd_places, nature_gpd, 'nature_index', 'gdf2')
avg_risk_3_places = calculate_avg_risk(gdf_sd_places, weather_gpd, 'Predicted_Fire_Risk', 'gdf3')
avg_risk_4_places = calculate_avg_risk(gdf_sd_places, life_gdf, 'psps_risk_scaled', 'gdf4')

gdf_sd_places = gdf_sd_places.join(avg_risk_1_places, how='left', rsuffix='_new1')
gdf_sd_places = gdf_sd_places.join(avg_risk_2_places, how='left', rsuffix='_new2')
gdf_sd_places = gdf_sd_places.join(avg_risk_3_places, how='left', rsuffix='_new3')
gdf_sd_places = gdf_sd_places.join(avg_risk_4_places, how='left', rsuffix='_new3')

# Create display columns for risk factors before filling NA
for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    disp_col = col + '_disp'
    gdf_sd_places[disp_col] = gdf_sd_places[col].apply(lambda x: f"{x} (missing)" if pd.isna(x) else x)
    gdf_sd_places[col] = gdf_sd_places[col].fillna(0)

# ---- Background Polygons ----
# Remove risk columns from gdf_sd if they exist
for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    if col in gdf_sd.columns:
        gdf_sd.drop(columns=[col], inplace=True)

avg_risk_1_sd = calculate_avg_risk(gdf_sd, energy_gpd, 'probability (%)', 'gdf1')
avg_risk_2_sd = calculate_avg_risk(gdf_sd, nature_gpd, 'nature_index', 'gdf2')
avg_risk_3_sd = calculate_avg_risk(gdf_sd, weather_gpd, 'Predicted_Fire_Risk', 'gdf3')
avg_risk_4_sd = calculate_avg_risk(gdf_sd, life_gdf, 'psps_risk_scaled', 'gdf4')

gdf_sd = gdf_sd.join(avg_risk_1_sd).join(avg_risk_2_sd).join(avg_risk_3_sd).join(avg_risk_4_sd)

# Create display columns for risk factors before filling NA
for col in ['avg_risk_gdf1', 'avg_risk_gdf2', 'avg_risk_gdf3']:
    disp_col = col + '_disp'
    gdf_sd[disp_col] = gdf_sd[col].apply(lambda x: f"{x} (missing)" if pd.isna(x) else x)
    gdf_sd[col] = gdf_sd[col].fillna(0)

for gdf in [gdf_sd_places, gdf_sd]:
    col = 'avg_risk_gdf4'
    disp_col = col + '_disp'
    gdf[disp_col] = gdf[col].apply(lambda x: f"{x} (missing)" if pd.isna(x) else x)
    gdf[col] = gdf[col].fillna(0)

gdf_sd_places['overall_risk'] = gdf_sd_places.apply(composite_risk, axis=1)
gdf_sd['overall_risk'] = gdf_sd.apply(composite_risk, axis=1)

# Remove incorporated places from background polygons
places_union = unary_union(gdf_sd_places.geometry)
gdf_sd['geometry'] = gdf_sd.geometry.apply(lambda geom: geom.difference(places_union))

# Remove any duplicated columns before converting to JSON
gdf_sd_places = gdf_sd_places.loc[:, ~gdf_sd_places.columns.duplicated()]
gdf_sd = gdf_sd.loc[:, ~gdf_sd.columns.duplicated()]

from branca.colormap import LinearColormap

gdf_sd = gdf_sd.reset_index(drop=True)
gdf_sd_places = gdf_sd_places.reset_index(drop=True)

# Combine overall risk values from both GeoDataFrames into one Series
combined_risk = pd.concat([gdf_sd['overall_risk'], gdf_sd_places['overall_risk']], ignore_index=True)

# Rank the combined series (lowest overall_risk gets rank 1)
combined_rank = combined_risk.rank(method='min', ascending=True)

# Assign the ranks back to each GeoDataFrame based on their positions in the combined series
gdf_sd['composite_rank'] = combined_rank.iloc[:len(gdf_sd)].values
gdf_sd_places['composite_rank'] = combined_rank.iloc[len(gdf_sd):].values

# For the colormap, set the rank bounds:
global_min_rank = 1
global_max_rank = int(combined_rank.max())

# Create a colormap that maps rank 1 (green) to rank global_max_rank (red)
colormap = LinearColormap(['red', 'green'], vmin=global_min_rank, vmax=global_max_rank)
colormap.caption = 'Composite Risk Rank'

# Define a custom heat style function that handles missing values and zero risk:
def heat_style(feature, fill_opacity=0.5):
    # Retrieve the rank property
    rank = feature['properties'].get('composite_rank')
    if rank is None or pd.isna(rank):
        fillColor = 'gray'
    else:
        fillColor = colormap(rank)
    return {
        'fillColor': fillColor,
        'color': 'black',
        'weight': 2,
        'fillOpacity': fill_opacity
    }

# ---- Create the Folium Map ----
center = [gdf_sd_places.geometry.unary_union.centroid.y,
          gdf_sd_places.geometry.unary_union.centroid.x]
m = folium.Map(location=center, zoom_start=10)
colormap.add_to(m)  # Add colormap to the map

# ---- Background Polygons ----
gdf_sd = gdf_sd.loc[:, ~gdf_sd.columns.duplicated()]

background_tooltip = folium.GeoJsonTooltip(
    fields=['NAME', 'avg_risk_gdf1_disp', 'avg_risk_gdf2_disp', 'avg_risk_gdf3_disp', 'avg_risk_gdf4_disp', 'overall_risk', 'composite_rank'],
    aliases=['Section:', 'Energy Risk:', 'Nature Risk:', 'Weather Risk:', 'Population Risk:', 'Overall Risk:', 'Rank:'],
    localize=True
)

folium.GeoJson(
    gdf_sd.to_json(),
    name='Background Polygons',
    tooltip=background_tooltip,
    style_function=lambda feature: heat_style(feature, fill_opacity=0.5)
).add_to(m)

# ---- Incorporated Places ----
gdf_sd_places = gdf_sd_places.loc[:, ~gdf_sd_places.columns.duplicated()]

places_tooltip = folium.GeoJsonTooltip(
    fields=['NAME_left', 'avg_risk_gdf1_disp', 'avg_risk_gdf2_disp', 'avg_risk_gdf3_disp', 'avg_risk_gdf4_disp', 'overall_risk', 'composite_rank'],
    aliases=['Place:', 'Energy Risk:', 'Nature Risk:', 'Weather Risk:', 'Population Risk:', 'Overall Risk:', 'Rank:'],
    localize=True
)

folium.GeoJson(
    gdf_sd_places.to_json(),
    name='Incorporated Places',
    tooltip=places_tooltip,
    style_function=lambda feature: heat_style(feature, fill_opacity=0.7)
).add_to(m)

folium.LayerControl().add_to(m)

# Finally, display the map (in a Jupyter Notebook, simply output m)
m

In [ ]:
m.save('result_viz.html')

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
from shapely import wkt
import matplotlib.pyplot as plt
import branca.colormap as cm

# ---- Define a style function based on risk rank ----
# Here, lower rank means higher risk. We want rank 1 to be red and highest rank to be green.
def risk_style(feature):
    rank = feature['properties']['risk_rank']
    return {
        'fillColor': colormap(rank),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    }

# ---------------------------
# 1. Load and prepare lines data
# ---------------------------
raw_df = pd.read_csv('data\\dev_wings_agg_span_2024_01_01.csv')
df_lines = raw_df[['shape', 'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
                     'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport', 'cust_total',
                     'downstream_cust_industrial', 'downstream_cust_commercial', 'downstream_cust_residential',
                     'downstream_cust_sensitive', 'downstream_cust_essential', 'downstream_cust_medicalcert',
                     'downstream_cust_urgent', 'downstream_cust_lifesupport', 'downstream_cust_total']]

# Convert the 'shape' column (WKT strings) to geometry objects
df_lines["geometry"] = df_lines["shape"].apply(wkt.loads)

# Set the initial CRS (assumed EPSG:2230) then reproject to EPSG:4326 for Folium
gdf_lines = gpd.GeoDataFrame(df_lines, geometry="geometry", crs="EPSG:2230")
gdf_lines = gdf_lines.to_crs("EPSG:4326")

# ---------------------------
# 2. Load and prepare polygon data
# ---------------------------
# Load subdivisions (background polygons)
subdiv_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUSUB/tl_2020_06_cousub.zip"
gdf_subdiv = gpd.read_file(subdiv_url)
gdf_sd = gdf_subdiv[gdf_subdiv['COUNTYFP'] == '073']
# Reproject background subdivisions to EPSG:4326
gdf_sd = gdf_sd.to_crs("EPSG:4326")

# Load incorporated places and counties
places_url = "https://www2.census.gov/geo/tiger/TIGER2020/PLACE/tl_2020_06_place.zip"
gdf_places = gpd.read_file(places_url)
counties_url = "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
gdf_counties = gpd.read_file(counties_url)
gdf_sd_county = gdf_counties[gdf_counties['NAME'] == 'San Diego']

# Reproject places and county to EPSG:4326
gdf_places = gdf_places.to_crs("EPSG:4326")
gdf_sd_county = gdf_sd_county.to_crs("EPSG:4326")

# Spatial join: Get incorporated places within San Diego County
gdf_sd_places = gpd.sjoin(gdf_places, gdf_sd_county, how="inner", predicate="intersects")
if 'index_right' in gdf_sd_places.columns:
    gdf_sd_places = gdf_sd_places.drop(columns=['index_right'])
gdf_sd_places = gdf_sd_places.loc[:, ~gdf_sd_places.columns.duplicated()]

# ---------------------------
# 3. Create non-overlapping background polygons
# ---------------------------
# Compute the union of the incorporated places
places_union = gdf_sd_places.unary_union

def subtract_places(geom, union_geom):
    return geom.difference(union_geom)

gdf_bg_nonoverlap = gdf_sd.copy()
gdf_bg_nonoverlap["geometry"] = gdf_bg_nonoverlap.geometry.apply(lambda g: subtract_places(g, places_union))
gdf_bg_nonoverlap = gdf_bg_nonoverlap[~gdf_bg_nonoverlap.is_empty]
gdf_bg_nonoverlap["layer"] = "Background"
gdf_sd_places["layer"] = "Incorporated Places"

# ---------------------------
# 4. Accumulate customer values into the non-overlapping polygons
# ---------------------------
gdf_combined = pd.concat([gdf_bg_nonoverlap, gdf_sd_places], ignore_index=True)

# Spatial join: count any overlapping line (using "intersects")
lines_in_polygons = gpd.sjoin(gdf_lines, gdf_combined, how="inner", predicate="intersects")

customer_fields = [
    'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
    'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport', 'cust_total', 
    'downstream_cust_industrial', 'downstream_cust_commercial', 'downstream_cust_residential',
    'downstream_cust_sensitive', 'downstream_cust_essential', 'downstream_cust_medicalcert',
    'downstream_cust_urgent', 'downstream_cust_lifesupport', 'downstream_cust_total'
]

accumulated = lines_in_polygons.groupby("index_right")[customer_fields].sum()
gdf_combined = gdf_combined.merge(accumulated, left_index=True, right_index=True, how='left')
gdf_combined[customer_fields] = gdf_combined[customer_fields].fillna(0)

# ---- Compute PSPS Risk Score and Rank ----
weights = {
    'cust_lifesupport': 10,
    'cust_medicalcert': 8,
    'cust_urgent': 6,
    'cust_sensitive': 4,
    'cust_industrial': 3,
    'cust_residential': 2,
    'cust_commercial': 1
}

gdf_combined['psps_risk'] = (
    weights['cust_lifesupport'] * (gdf_combined['cust_lifesupport'] + gdf_combined['downstream_cust_lifesupport']) +
    weights['cust_medicalcert'] * (gdf_combined['cust_medicalcert'] + gdf_combined['downstream_cust_medicalcert']) +
    weights['cust_urgent'] * (gdf_combined['cust_urgent'] + gdf_combined['downstream_cust_urgent']) +
    weights['cust_sensitive'] * (gdf_combined['cust_sensitive'] + gdf_combined['downstream_cust_sensitive']) +
    weights['cust_industrial'] * (gdf_combined['cust_industrial'] + gdf_combined['downstream_cust_industrial']) +
    weights['cust_residential'] * (gdf_combined['cust_residential'] + gdf_combined['downstream_cust_residential']) +
    weights['cust_commercial'] * (gdf_combined['cust_commercial'] + gdf_combined['downstream_cust_commercial'])
)

# Rank polygons by risk: rank 1 = highest risk.
gdf_combined['risk_rank'] = gdf_combined['psps_risk'].rank(method='min', ascending=False).astype(int)

# ---------------------------
# 5. Create the Folium Map using heatmap styling (based on risk rank)
# ---------------------------
gdf_combined_bg = gdf_combined[gdf_combined["layer"] == "Background"]
gdf_combined_places = gdf_combined[gdf_combined["layer"] == "Incorporated Places"]

# Build a colormap based on risk_rank. Since rank 1 is highest risk, we want rank 1 to be red.
min_rank = gdf_combined['risk_rank'].min()  # typically 1
max_rank = gdf_combined['risk_rank'].max()
colormap = cm.LinearColormap(colors=["red", "green"], vmin=min_rank, vmax=max_rank)
colormap.caption = "PSPS Risk Rank (1 = Highest Risk)"

# Compute map center based on incorporated places’ union
center = [gdf_sd_places.geometry.unary_union.centroid.y,
          gdf_sd_places.geometry.unary_union.centroid.x]
m = folium.Map(location=center, zoom_start=10)
colormap.add_to(m)

# ---- Background Polygons ----
background_tooltip = folium.GeoJsonTooltip(
    fields=['NAME', 'cust_total', 'downstream_cust_total', 'risk_rank'],
    aliases=['Section:', 'Customer Total:', 'Downstream Total:', 'Population Risk Rank:'],
    localize=True
)

folium.GeoJson(
    gdf_combined_bg.to_json(),
    name='Background Polygons',
    tooltip=background_tooltip,
    style_function=risk_style
).add_to(m)

# ---- Incorporated Places ----
places_tooltip = folium.GeoJsonTooltip(
    fields=['NAME_left', 'cust_total', 'downstream_cust_total', 'risk_rank'],
    aliases=['Place:', 'Customer Total:', 'Downstream Total:', 'Population Risk Rank:'],
    localize=True
)

folium.GeoJson(
    gdf_combined_places.to_json(),
    name='Incorporated Places',
    tooltip=places_tooltip,
    style_function=risk_style
).add_to(m)

folium.LayerControl().add_to(m)

# ---------------------------
# 6. Save the accumulated polygon data to CSV
# ---------------------------
gdf_combined["geometry"] = gdf_combined["geometry"].apply(lambda x: x.wkt)
gdf_combined.to_csv("accumulated_nonoverlap_polygon_values.csv", index=False)

m.save("nonoverlap_polygons_map.html")
m

In [ ]:
m.save("result_viz.html")